<a href="https://colab.research.google.com/github/SomeiLam/langchain-example/blob/main/LangChain_evaluation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# LangChain: Evaluation

## Outline:

* Example generation
* Manual evaluation (and debuging)
* LLM-assisted evaluation
* LangChain evaluation platform

In [ ]:
!pip install langchain
!pip install openai
!pip install langchain_community
!pip install -U langchain-openai
!pip install docarray
!pip install -U langchain-openai

In [47]:
import os
import openai
from google.colab import userdata
api_key = userdata.get("OPENAI_API_KEY")
os.environ["OPENAI_API_KEY"] = api_key
openai.api_key = os.environ['OPENAI_API_KEY']
llm_model = "gpt-3.5-turbo"

In [8]:
from langchain.chains import RetrievalQA
from langchain_openai import ChatOpenAI
from langchain.document_loaders import CSVLoader
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

In [9]:
file = 'OutdoorClothingCatalog_1000.csv'
loader = CSVLoader(file_path=file)
data = loader.load()

In [13]:
from langchain_openai import OpenAIEmbeddings

embedding_model = OpenAIEmbeddings()

In [11]:
from langchain.indexes import VectorstoreIndexCreator
from langchain.vectorstores import DocArrayInMemorySearch

index_creator = VectorstoreIndexCreator(
    embedding=embedding_model,
    vectorstore_cls=DocArrayInMemorySearch,
)
index = index_creator.from_loaders([loader])

/usr/local/lib/python3.11/dist-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [48]:
llm = ChatOpenAI(temperature = 0.0, model=llm_model)
qa = RetrievalQA.from_chain_type(
    llm=llm,
    chain_type="stuff",
    retriever=index.vectorstore.as_retriever(),
    verbose=True,
    chain_type_kwargs = {
        "document_separator": "<<<<>>>>>"
    }
)

In [14]:
data[10]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 10}, page_content=": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-on pants have a wide elastic waistband and drawstring, side pockets and a modern slim leg.\n\nImported.")

In [15]:
data[11]

Document(metadata={'source': 'OutdoorClothingCatalog_1000.csv', 'row': 11}, page_content=': 11\nname: Ultra-Lofty 850 Stretch Down Hooded Jacket\ndescription: This technical stretch down jacket from our DownTek collection is sure to keep you warm and comfortable with its full-stretch construction providing exceptional range of motion. With a slightly fitted style that falls at the hip and best with a midweight layer, this jacket is suitable for light activity up to 20° and moderate activity up to -30°. The soft and durable 100% polyester shell offers complete windproof protection and is insulated with warm, lofty goose down. Other features include welded baffles for a no-stitch construction and excellent stretch, an adjustable hood, an interior media port and mesh stash pocket and a hem drawcord. Machine wash and dry. Imported.')

### Hard-coded examples

In [16]:
examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]

### LLM-Generated examples

In [17]:
from langchain.evaluation.qa import QAGenerateChain

In [20]:
llm = ChatOpenAI(model="gpt-3.5-turbo", temperature=0)
example_gen_chain = QAGenerateChain.from_llm(llm)

In [25]:
# Prepare inputs as a list of dicts (each must have a "doc" key)
inputs = [{"doc": t} for t in data[:5]]

# 2) Apply to get raw LLM outputs
inputs      = [{"doc": t} for t in data[:5]]
raw_results = example_gen_chain.apply(inputs)
# raw_results is a list of dicts: [{"text": "Q1: …\nA1: …"}, …]
raw_results


[{'qa_pairs': {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
   'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."}},
 {'qa_pairs': {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
   'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'}},
 {'qa_pairs': {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece have according to the document?",
   'answer': 'The swimsuit features bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and is machine washable and line dry for best results.'}},
 {'qa_pairs': {'query': 'What is the fabric composition of the Refresh Swimwear V-Neck Tankini Contrasts?',
   'answer': 'The body of t

In [39]:
flat_qas = [item["qa_pairs"] for item in raw_results]
queries = [qa["query"]   for qa in flat_qas]
answers = [qa["answer"] for qa in flat_qas]
flat_qas

[{'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece have according to the document?",
  'answer': 'The swimsuit features bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and is machine washable and line dry for best results.'},
 {'query': 'What is the fabric composition of the Refresh Swimwear V-Neck Tankini Contrasts?',
  'answer': 'The body of the tankini is made of 82% recycled nylon and 18% Lycra® spa

In [35]:
import pandas as pd

df_qas = pd.DataFrame(flat_qas)

[{'query': "What is the approximate weight of the Women's Campside Oxfords per pair?",
  'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."},
 {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?',
  'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".'},
 {'query': "What features does the Infant and Toddler Girls' Coastal Chill Swimsuit, Two-Piece have according to the document?",
  'answer': 'The swimsuit features bright colors, ruffles, exclusive whimsical prints, four-way-stretch and chlorine-resistant fabric, UPF 50+ rated fabric for sun protection, crossover no-slip straps, fully lined bottom, and is machine washable and line dry for best results.'},
 {'query': 'What is the fabric composition of the Refresh Swimwear V-Neck Tankini Contrasts?',
  'answer': 'The body of the tankini is made of 82% recycled nylon and 18% Lycra® spa

In [49]:
examples += flat_qas
query = examples[0]["query"]
result = qa.invoke(query)



> Entering new RetrievalQA chain...

> Finished chain.


In [50]:
result

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

## Manual Evaluation

In [51]:
import langchain
langchain.debug = True

In [52]:
qa.invoke(examples[0]["query"])

[chain/start] [chain:RetrievalQA] Entering Chain run with input:
{
  "query": "Do the Cozy Comfort Pullover Set        have side pockets?"
}
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain] Entering Chain run with input:
[inputs]
[chain/start] [chain:RetrievalQA > chain:StuffDocumentsChain > chain:LLMChain] Entering Chain run with input:
{
  "question": "Do the Cozy Comfort Pullover Set        have side pockets?",
  "context": ": 10\nname: Cozy Comfort Pullover Set, Stripe\ndescription: Perfect for lounging, this striped knit set lives up to its name. We used ultrasoft fabric and an easy design that's as comfortable at bedtime as it is when we have to make a quick run out.\n\nSize & Fit\n- Pants are Favorite Fit: Sits lower on the waist.\n- Relaxed Fit: Our most generous fit sits farthest from the body.\n\nFabric & Care\n- In the softest blend of 63% polyester, 35% rayon and 2% spandex.\n\nAdditional Features\n- Relaxed fit top with raglan sleeves and rounded hem.\n- Pull-

{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?',
 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}

In [53]:
# Turn off the debug mode
langchain.debug = False

## LLM assisted evaluation

In [57]:
new_examples = [
    {
        "query": "Do the Cozy Comfort Pullover Set\
        have side pockets?",
        "answer": "Yes"
    },
    {
        "query": "What collection is the Ultra-Lofty \
        850 Stretch Down Hooded Jacket from?",
        "answer": "The DownTek collection"
    }
]
new_examples += flat_qas

In [58]:
predictions = qa.batch(new_examples)



> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...


> Entering new RetrievalQA chain...

> Finished chain.


> Entering new RetrievalQA chain...

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.

> Finished chain.


In [59]:
from langchain.evaluation.qa import QAEvalChain

In [60]:
llm = ChatOpenAI(temperature=0, model=llm_model)
eval_chain = QAEvalChain.from_llm(llm)

In [62]:
graded_outputs = eval_chain.evaluate(new_examples, predictions)

In [71]:
for i, eg in enumerate(new_examples):
    print(predictions)
    print(f"Example {i}:")
    print("Question:        " + predictions[i]["query"])
    print("Real Answer:     " + predictions[i]["answer"])
    print("Predicted Answer:" + predictions[i]["result"])
    print("Predicted Grade: " + graded_outputs[i]["results"])
    print()

[{'query': 'Do the Cozy Comfort Pullover Set        have side pockets?', 'answer': 'Yes', 'result': 'Yes, the Cozy Comfort Pullover Set does have side pockets.'}, {'query': 'What collection is the Ultra-Lofty         850 Stretch Down Hooded Jacket from?', 'answer': 'The DownTek collection', 'result': 'The Ultra-Lofty 850 Stretch Down Hooded Jacket is from the DownTek collection.'}, {'query': "What is the approximate weight of the Women's Campside Oxfords per pair?", 'answer': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz.", 'result': "The approximate weight of the Women's Campside Oxfords per pair is 1 lb. 1 oz."}, {'query': 'What are the dimensions of the small and medium sizes of the Recycled Waterhog Dog Mat, Chevron Weave?', 'answer': 'The small size has dimensions of 18" x 28" and the medium size has dimensions of 22.5" x 34.5".', 'result': 'The dimensions of the small size of the Recycled Waterhog Dog Mat, Chevron Weave are 18" x 28", and the dime

In [72]:
graded_outputs[0]

{'results': 'CORRECT'}